In [2]:
import math
import requests
import re

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from bs4 import BeautifulSoup

In [3]:
base_url = "https://data.bka.gv.at/ris/api/v2.6/Landesrecht"
params = {
    "Applikation": "LrKons",
    "Bundesland_SucheInSalzburg": "true",
    "Sortierung_SortDirection": "Ascending",
    "DokumenteProSeite": "Ten",
    "Seitennummer": 1,
    "Fassung_FassungVom": "2025-06-09"
}

resp = requests.get(base_url, params=params)
resp.raise_for_status()
data = resp.json()

hits = int(data["OgdSearchResult"]["OgdDocumentResults"]["Hits"]["#text"])
page_size = int(data["OgdSearchResult"]["OgdDocumentResults"]["Hits"]["@pageSize"])
total_pages = math.ceil(hits / page_size)
print(f"Gesamt-Treffer: {hits}, Seitengröße: {page_size}, Gesamt-Seiten: {total_pages}")

Gesamt-Treffer: 13601, Seitengröße: 10, Gesamt-Seiten: 1361


In [7]:
all_docs = []
total_pages = 1
for page in range(1, total_pages + 1):
    print(f"Lade Seite {page}/{total_pages}")
    params["Seitennummer"] = page
    resp = requests.get(base_url, params=params)
    resp.raise_for_status()
    data = resp.json()
    docs = data["OgdSearchResult"]["OgdDocumentResults"]["OgdDocumentReference"]
    if isinstance(docs, list):
        all_docs.extend(docs)
    else:
        all_docs.append(docs)

print(f"Anzahl gesammelter Dokument-Referenzen: {len(all_docs)}")

all_texts = []
for doc in all_docs:
    lrKons = doc["Data"]["Metadaten"]["Landesrecht"]["LrKons"]
    url = lrKons.get("GesamteRechtsvorschriftUrl")
    text = ""

    if url:
        try:
            page_resp = requests.get(url)
            page_resp.raise_for_status()
            soup = BeautifulSoup(page_resp.text, "html.parser")
            header = soup.find("h3", string=re.compile(r'\bText\b', re.IGNORECASE))
            if header:
                content = soup.find("div", class_="document")
                text = content.get_text(separator="\n", strip=True)
        except requests.RequestException:
            pass

    if not text:
        text = "Rechtsvorschrift noch nicht dokumentiert."

    all_texts.append(text)

print(f"Anzahl gesammelter Text-Passagen: {len(all_texts)}")
print("Erster Text (gekürzt):\n", all_texts[0][:300], "…")
print()
print("Alle Texte:")
for x, text in enumerate(all_texts, start=1):
    print(f"\n--- Dokument {x} ---\n")
    print(text)

Lade Seite 1/1
Anzahl gesammelter Dokument-Referenzen: 10
Anzahl gesammelter Text-Passagen: 10
Erster Text (gekürzt):
 Rechtsvorschrift noch nicht dokumentiert. …

Alle Texte:

--- Dokument 1 ---

Rechtsvorschrift noch nicht dokumentiert.

--- Dokument 2 ---

Rechtsvorschrift noch nicht dokumentiert.

--- Dokument 3 ---

Rechtsvorschrift noch nicht dokumentiert.

--- Dokument 4 ---

§ 0
Langtitel
Verordnung der Salzburger Landesregierung vom 3. Dezember 2001 zur Anpassung diverser Verordnungen an die Umstellung von der Schilling- auf die Eurowährung(1. Euro-Begleit-Verordnung)
StF:
LGBl Nr 111/2001
Präambel/Promulgationsklausel
Auf Grund der Gesetze, auf deren Grundlage die im Folgenden genannten Verordnungen der Landesregierung erlassen worden sind, wird verordnet:
Inhaltsverzeichnis
Artikel  1 Satzung der Salzburger Landes-Holding
Artikel  2 Gemeindehaushaltsverordnung 1998
Artikel  3 Bürgermeister-Entschädigungs-Verordnung
Artikel  4 Entschädigungs-Verordnung
Artikel  5 Pauschaliert